In [1]:
import cProfile
from io import StringIO
import pstats

import matplotlib.pyplot as plt
import numpy as np

from superfv import AdvectionSolver
import superfv.initial_conditions as ic
import superfv.visualization as vis

In [2]:
N = 1024
p = 0

sim = AdvectionSolver(
    ic=lambda idx, x, y, z: ic.square(idx, x, y, z, vx=2, vy=1),
    nx=N,
    ny=N,
    p=p,
    interpolation_scheme="transverse",
    cupy=True,
    log_every_step=False,
)

In [3]:
sim.run(n=5)

100%|██████████| 5/5 [00:00<00:00, 26.28it/s]


In [ ]:
pr = cProfile.Profile()
pr.enable()
sim.run(n=5)
pr.disable()

s = StringIO()
ps = pstats.Stats(pr, stream=s).sort_stats("cumtime")
ps.print_stats()

with open("profile_output.out", "w") as f:
    f.write(s.getvalue())

In [ ]:
sim.timer.print_report()

In [ ]:
fig, axs = plt.subplots(2, 1, sharex=True)

axs[1].set_xlabel("Step #")
axs[0].set_ylabel("Run time (s)")
axs[1].set_ylabel("# Updates/s")
axs[0].set_yscale("log")
axs[1].set_yscale("log")

axs[0].plot(
    sim.minisnapshots["run_time"][1:], linestyle="--", marker="*", label=f"{p=}, {N=}"
)
axs[1].plot(
    np.array(sim.minisnapshots["n_updates"])[1:]
    / np.array(sim.minisnapshots["run_time"])[1:],
    linestyle="--",
    marker="*",
    label=f"{p=}, {N=}",
)

axs[0].legend()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
vis.plot_1d_slice(sim, ax, "rho", cell_averaged=True, x=None, t=0, color="k")
vis.plot_1d_slice(sim, ax, "rho", cell_averaged=True, x=None, xlabel=True)